In [1]:

import os, sys
import cftime
import datetime
import xarray as xr
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add the directory containing 'cmct' to the Python path
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.insert(0, cmct_dir)
from cmct.imbie import *

# Use a writable directory inside your home path
upload_directory = cmct_dir+'/uploaded_file'

output=widgets.Output()
upload_output = widgets.Output()
process_output = widgets.Output()
result_output = widgets.Output()

# Ensure the directory exists
os.makedirs(upload_directory, exist_ok=True)

global nc_filename

models=['GIS','AIS']
# Global variable to store the uploaded file path
icesheet_widget = widgets.Dropdown(
    options=models,
    value='GIS',
    description='Ice Sheet:'
)

#File Upload Widget
upload_widget = widgets.FileUpload(
    accept='.nc',
    multiple=False
)
upload_widget.disabled = (icesheet_widget.value == 'None')

#Start Date Picker
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    disabled=True
)

#End Date Picker
end_date_widget = widgets.DatePicker(
    description='End Date:',
    disabled=True
)

#Mass Balance Dropdown
mass_balance_widget = widgets.Dropdown(
    options=[
        ('Cumulative mass balance (Gt)', 'total'),
        ('Cumulative dynamics mass balance anomaly (Gt)', 'dynamic')
    ],
    value='total',
    description='Mass Balance:',
    disabled=True,
    style={'description_width': 'initial'}
)

#Run Process Button
run_button = widgets.Button(
    description="Run Process",
    disabled=True
)

#Function to update file paths AFTER a file is uploaded
def update_file_paths():
    global projection, shape_filename, obs_filename, obs_east_filename, obs_west_filename, obs_peninsula_filename

    icesheet = icesheet_widget.value  # Get the currently selected Ice Sheet
    with output:
        clear_output(wait=True)

    if icesheet == 'GIS':
        projection = 'EPSG:3413'
        #nc_filename = uploaded_nc_file  # Use the uploaded file
        shape_filename = cmct_dir + '/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp'
        obs_filename = cmct_dir + '/data/IMBIE/imbie_greenland_2021_Gt.csv'
        obs_east_filename, obs_west_filename, obs_peninsula_filename = None, None, None

    elif icesheet == 'AIS':
        projection = 'EPSG:3031'
        #nc_filename = uploaded_nc_file  # Use the uploaded file
        shape_filename = cmct_dir + '/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp'
        obs_filename = cmct_dir + '/data/IMBIE/imbie_antarctica_2021_Gt.csv'
        obs_east_filename, obs_west_filename, obs_peninsula_filename = None, None, None

#Function to handle Ice Sheet selection
def on_icesheet_change(change):
    
    # Enable upload button after GIS/AIS selection
    upload_widget.disabled = (change['new'] == 'None') 
    update_file_paths()


#Function to handle file upload
def on_upload_change_orig(change):
    upload_output.clear_output(wait=True)
    with upload_output:
        clear_output(wait=True)
        global uploaded_nc_file
        global nc_filename
        uploaded_nc_file = None  # Reset previous file

        if not upload_widget.value:
            with upload_output:
                print("No file detected in FileUpload widget!")
            return

        for file in os.listdir(upload_directory):
            file_path = os.path.join(upload_directory, file)
            try:
                os.remove(file_path)
            except Exception as e:
                with upload_output:
                    print(f"Error deleting file {file_path}: {e}")

        try:
            # Extract file content correctly
            uploaded_file = list(upload_widget.value.values())[0]  # Extract file dict
            file_name = uploaded_file['metadata']['name']  # Get file name
            file_content = uploaded_file['content']  # Get file binary content
            file_path = os.path.join(upload_directory, file_name)  # Target path

            # Write file to the specified directory
            with open(file_path, "wb") as f:
                f.write(file_content)

            print('File saved successfully')
            uploaded_nc_file = file_path  # Store path globally
            nc_filename = uploaded_nc_file

            update_file_paths()

            mod_ds = xr.open_dataset(uploaded_nc_file, use_cftime=True)
            time_var = mod_ds['time']
            min_time = time_var.values.min()
            max_time = time_var.values.max()
            print(f"Model data time range is from {min_time} to {max_time}.Choose dates accordingly")

            start_date_widget.disabled = False

        except Exception as e:
            with upload_output:
                clear_output(wait=True)
                print(f"Error during file upload: {e}")

def on_upload_change(change):
    upload_output.clear_output(wait=True)
    try:
        for file in os.listdir(upload_directory):
            file_path = os.path.join(upload_directory, file)
            try:
                os.remove(file_path)
            except Exception as e:
                with upload_output:
                    print(f"Error deleting file {file_path}: {e}")

        uploaded_file = list(upload_widget.value.values())[0]
        file_name = uploaded_file['metadata']['name']
        file_content = uploaded_file['content']
        file_path = os.path.join(upload_directory, file_name)

        with open(file_path, "wb") as f:
            f.write(file_content)

        global uploaded_nc_file, nc_filename
        uploaded_nc_file = file_path
        nc_filename = uploaded_nc_file

        update_file_paths()

        mod_ds = xr.open_dataset(uploaded_nc_file, use_cftime=True)
        time_var = mod_ds['time']
        min_time = time_var.values.min()
        max_time = time_var.values.max()

        with upload_output:
            print("File saved successfully")
            print(f"Model data time range is from {min_time} to {max_time}. Choose dates accordingly")

        start_date_widget.disabled = False

    except Exception as e:
        with upload_output:
            clear_output(wait=True)
            print(f"Error during file upload: {e}")


# Function to enable end date after selecting start date
def on_start_date_change(change):
    with output:
        clear_output(wait=True)
    if start_date_widget.value:
        end_date_widget.disabled = False  

# Function to enable mass balance dropdown after selecting end date
def on_end_date_change(change):
    with output:
        clear_output(wait=True)
    if end_date_widget.value:  # Check if a date is selected
        mass_balance_widget.disabled = False
        if mass_balance_widget.value:
            run_button.disabled = False

# Function to enable run button after selecting mass balance
def on_mass_balance_change(change):
    with output:
        clear_output(wait=True)
    if mass_balance_widget.value:
        run_button.disabled = False  


# Attach event listeners to widgets
icesheet_widget.observe(on_icesheet_change, names='value')
upload_widget.observe(on_upload_change, names='value')
start_date_widget.observe(on_start_date_change, names='value')
end_date_widget.observe(on_end_date_change, names='value')
mass_balance_widget.observe(on_mass_balance_change, names='value')

display(icesheet_widget)
display(upload_widget)
display(upload_output)
display(start_date_widget)
display(end_date_widget)
display(mass_balance_widget)
display(run_button)


Dropdown(description='Ice Sheet:', index=1, options=('None', 'GIS', 'AIS'), value='GIS')

FileUpload(value={}, accept='.nc', description='Upload')

DatePicker(value=None, description='Start Date:', disabled=True)

DatePicker(value=None, description='End Date:', disabled=True)

Dropdown(description='Mass Balance:', disabled=True, options=(('Cumulative mass balance (Gt)', 'total'), ('Cum…

Button(description='Run Process', disabled=True, style=ButtonStyle())

file_name is: lithk_GIS_GSFC_ISSM_historical.nc


Output()

In [ ]:
# Check if observation file exists
def check_files():
    if not os.path.exists(obs_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_filename}")

    # Check if model file exists    
    if not os.path.exists(nc_filename):
        raise FileNotFoundError(f"Model file not found: {nc_filename}")
    
    icesheet = icesheet_widget.value
    
    if icesheet == 'AIS':
        if (obs_east_filename and os.path.exists(obs_east_filename)) and \
               (obs_west_filename and os.path.exists(obs_west_filename)) and \
               (obs_peninsula_filename and os.path.exists(obs_peninsula_filename)):
            # Check if regional observation files exist 
            if not os.path.exists(obs_east_filename):
                raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
            if not os.path.exists(obs_west_filename):
                raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
            if not os.path.exists(obs_peninsula_filename):
                raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

In [ ]:
import time
display(process_output)
# Function to run the process when all selections are made
def on_run_process_clicked(b):
    process_output.clear_output(wait=True)
    with process_output:
        clear_output(wait=True)
        check_files()
        run_button.disabled = True
        print("Running the process...")
        #time.sleep(0.5)
        

        # Convert start/end comparison times to fractional year
        try:
            # Open model file
            mod_ds = xr.open_dataset(uploaded_nc_file, use_cftime=True)
            time_var = mod_ds['time']
            calendar_type = time_var.to_index().calendar
            start_date_dt = start_date_widget.value
            end_date_dt = end_date_widget.value

            # Adjust day to avoid errors in non-Gregorian calendars
            start_date_cftime = cftime.datetime(start_date_dt.year, start_date_dt.month, min(start_date_dt.day, 30), calendar=calendar_type)
            end_date_cftime = cftime.datetime(end_date_dt.year, end_date_dt.month, min(end_date_dt.day, 30), calendar=calendar_type)

            start_date_fract = start_date_cftime.year + (start_date_cftime.dayofyr - 1) / 365
            end_date_fract = end_date_cftime.year + (end_date_cftime.dayofyr - 1) / 365

            # Mapping of user-friendly options to actual column names
            mass_balance_column_mapping = {
                'total': 'Cumulative mass balance (Gt)',
                'dynamic': 'Cumulative dynamics mass balance anomaly (Gt)'
            }

            # Get the correct column name based on user selection
            mass_balance_type = mass_balance_widget.value
            mass_balance_column = mass_balance_column_mapping[mass_balance_type]

            IMBIE_mass_change = process_imbie_data(obs_filename, start_date_fract, end_date_fract, mass_balance_column)

            model_mass_change = process_model_data(
                mod_ds, time_var, IMBIE_mass_change, start_date_cftime, end_date_cftime, start_date_fract, end_date_fract, 918, projection, shape_filename, icesheet_widget.value
            )

            imbie_model_residuals = calculate_model_imbie_residuals(
                start_date_fract, end_date_fract, icesheet_widget.value, model_mass_change, IMBIE_mass_change, mass_balance_type, None, None, None
            )

            # Extract the base name of the uploaded .nc file (without extension)
            nc_base_filename = os.path.basename(uploaded_nc_file).replace('.nc', '')

            # Define the output directory
            output_path = os.path.join(cmct_dir, 'data/')
            os.makedirs(output_path, exist_ok=True)  # Ensure directory exists

            # Create the CSV filename
            csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")

            clear_output(wait=True)
            write_and_display_mass_change_comparison_all_dates(
                icesheet_widget.value, model_mass_change, imbie_model_residuals, mass_balance_type, start_date_fract, end_date_fract, csv_filename
            )

            print(f"Process completed successfully! Output saved to: {csv_filename}")
        except Exception as e:
            #if time_var.values.min() <= start_date_cftime <= time_var.values.max() and time_var.values.min() <= end_date_cftime <= time_var.values.max():
                print(f"Error: {e}") 
        finally:
            run_button.disabled = False

run_button.on_click(on_run_process_clicked)